* Lựa chọn các thông tin có giá trị (xóa các cột không cần thiết)
    + Cột đầu tiên của tất cả các bảng
    + food: item
    + menu: cuisine
    + order: currency (xóa cả dòng có currency = USD)
    + restaurant: lic_no, link, address, menu, rating_count
    + users: name, email, password

* Chuẩn hóa dữ liệu
    + food: veg_non_veg -> 0,1
    + orders: sales_amount -> giá trị âm
    + restaurant: cost -> xóa ký tự rup, cuisine -> tách ra
    + users: Monthly Income -> 0, 1, 2

* Rút gọn dữ liệu (tổng hợp số liệu theo tháng) (optinal)


In [2]:
import pandas as pd

food = pd.read_csv('dataset/food_n.csv', sep='\t')
menu = pd.read_csv('dataset/menu_n.csv', on_bad_lines='skip', sep='\t')
orders = pd.read_csv('dataset/orders_n.csv', sep='\t')
restaurant = pd.read_csv('dataset/restaurant_n.csv', on_bad_lines='skip', sep='\t')
users = pd.read_csv('dataset/users_n.csv', sep='\t')


C:\Users\DUY LINH\AppData\Local\Temp\ipykernel_16424\2619362460.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  menu = pd.read_csv('dataset/menu_n.csv', on_bad_lines='skip', sep='\t')


In [10]:
# food.info()
menu.info()
# orders.info()
# restaurant.info()
# users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1179438 entries, mn0	567335	fd0	Beverages to mn1048562	96525	fd1048562	Ice Cream
Data columns (total 1 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   menu_id	r_id	f_id	cuisine	price  938315 non-null  object
dtypes: object(1)
memory usage: 18.0+ MB


In [4]:
print('Shape of food table:         ({}, {})'.format(food.shape[0], food.shape[1]))
print('Shape of menu table:         ({}, {})'.format(menu.shape[0], menu.shape[1]))
print('Shape of orders table:       ({}, {})'.format(orders.shape[0], orders.shape[1]))
print('Shape of restaurant table:   ({}, {})'.format(restaurant.shape[0], restaurant.shape[1]))
print('Shape of users table:        ({}, {})'.format(users.shape[0], users.shape[1]))

Shape of food table:         (371560, 2)
Shape of menu table:         (1179936, 5)
Shape of orders table:       (147061, 5)
Shape of restaurant table:   (148255, 7)
Shape of users table:        (100000, 8)


Xóa dữ liệu có giá trị bất kỳ null, giá trị trùng lặp

In [168]:
users = users.dropna()
users.drop_duplicates(keep=False, inplace=True)

Normalization 

In [177]:
food_menu = pd.merge(food, menu, on='f_id', how='inner')

In [304]:
restaurant['rating'] = pd.to_numeric(restaurant['rating'], errors='coerce').astype('float64')

In [ ]:
restaurant['rating'] = restaurant['rating'].map({'--': restaurant['rating'].mean()})

Điền giá trị trung bình vào các dòng có rating là rỗng

In [305]:
restaurant['rating'] = restaurant['rating'].map({'--': restaurant['rating'].mean()})
restaurant['rating'].fillna((restaurant['rating'].mean()), inplace=True)

Xóa các đơn đặt hàng có doanh thu âm, thanh toán bằng USD

In [ ]:
orders = orders[orders['sales_amount'] > 0]
orders = orders[orders['currency'] != 'USD']

In [312]:
restaurant["cost"] = restaurant["cost"].str.replace("₹ ", "").astype(int)

geopy


In [ ]:
restaurant['city'].sort_values().unique()

array(['', 'Andhra Pradesh', 'Arunachal Pradesh', 'Assam',
       'Auvergne-Rhône-Alpes', 'Banda', 'Bihar', 'Centre-Val de Loire',
       'Chhattisgarh', 'Dadra and Nagar Haveli and Daman and Diu',
       'Delhi', 'Goa', 'Gujarat', 'Haryana', 'Himachal Pradesh',
       'Jammu and Kashmir', 'Jawa Tengah', 'Jharkhand', 'Karnataka',
       'Kerala', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Meghalaya',
       'Mizoram', 'Nagaland', 'Nouvelle-Aquitaine', 'Occitanie', 'Odisha',
       'Oregon', 'Puducherry', 'Punjab', 'Rajasthan', 'Sardigna/Sardegna',
       'Sikkim', 'Tamil Nadu', 'Telangana', 'Tripura', 'Uttar Pradesh',
       'Uttarakhand', 'West Bengal', 'Île-de-France', 'استان خراسان رضوی'],
      dtype=object)

Lấy tên bang từ thông tin địa chỉ, tên thành phố

In [ ]:
formated_city = []
wrong_format_city = {'Bapatlachirala': 'Chirala', '':'Uttar Pradesh', 
                     'Bharabanki':'Barabanki', 'Bodhan-rural':'Bodhan',
                     'Bongaon':'Bangaon', 'Karunagappaly':'Karunagappally',
                     'Kothamanagalam':'Kothamangalam', 'Kunnamkullam':'Kunnamkulam',
                     'Surendranagar-dudhrej':'Surendranagar', 'Banda':'Uttar Pradesh'}

for index, row in restaurant.iterrows():
    city = row['city']
    
    if city not in formated_city:
    
        last_city = city.rpartition(',')[-1]
        
        restaurant.loc[restaurant['city'] == city, 'city'] = last_city
        formated_city.extend([city])
        
    for key, value in wrong_format_city.items():
        if city == key:
            restaurant.loc[restaurant['city'] == city, 'city'] = value

In [ ]:
from geopy.geocoders import Nominatim

formated_state_city = []
geolocator = Nominatim(user_agent="http")

for index, row in restaurant.iterrows():
    city = row['city']

    if city not in formated_state_city:
        location = geolocator.geocode(city, addressdetails=True)

        full_address = location.raw['address']
        state = full_address.get('state', '')

        restaurant.loc[restaurant['city'] == city, 'city'] = state
        formated_state_city.extend([city])

Xuất data đã được chuẩn hóa

In [ ]:
food.to_csv('dataset/food_n.csv', 
                  columns=['f_id', 'veg_or_non_veg']
                  , sep='\t', index=False)

menu.to_csv('dataset/menu_n.csv', 
                  columns=['menu_id', 'r_id', 'f_id', 'cuisine', 'price']
                  , sep='\t', index=False)

restaurant.to_csv('dataset/restaurant_n.csv', 
                  columns=['id', 'city', 'rating', 'rating_count'
                           , 'cost', 'cuisine']
                  , sep='\t', index=False)

orders.to_csv('dataset/orders_n.csv', 
                  columns=['order_date', 'sales_qty', 'sales_amount'
                           , 'user_id', 'cur_idisine']
                  , sep='\t', index=False)

users.to_csv('dataset/users_n.csv', 
                  columns=['user_id', 'Age', 'Gender', 'Marital'
                           , 'Status', 'Occupation', 'Monthly Income'
                           , 'Educational', 'Qualifications'
                           , 'Family size']
                  , sep='\t', index=False)